In [221]:
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, concatenate, Dense, Dropout
from keras import Model
from keras.callbacks import TensorBoard, EarlyStopping
import keras.callbacks as kc
import keras.backend as K

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import time
import numpy as np
import sys
import os

sys.path.append(os.path.join(os.getcwd(),"Preprocess"))
sys.path.append(os.path.join(os.getcwd(),"DataAcquisition"))

from helper import *
from transformers import *

init_config()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def get_dummy_data():
    X_a = np.ones((1000, 100, 146, 1))
    y_a = np.array([[1,0]]*X_a.shape[0])

    X_b = np.zeros((1000, 100, 146, 1))
    y_b = np.array([[0,1]]*X_b.shape[0])

    X = np.vstack((X_a,X_b))
    y = np.concatenate((y_a,y_b))

    return train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

In [144]:
def InceptionModule1(x, nr_filters=16):
    tower_1 = Conv2D(nr_filters, (1, x.shape[2].value), padding='same', activation='relu')(x)
    tower_2 = Conv2D(nr_filters, (3, x.shape[2].value), padding='same', activation='relu')(x)
    tower_3 = Conv2D(nr_filters, (5, x.shape[2].value), padding='same', activation='relu')(x)
    
    return concatenate([tower_1, tower_2, tower_3], axis=3)
    

def InceptionModule2(x, nr_filters = 16):
    tower_1 = Conv2D(nr_filters, (1, x.shape[2].value), padding='same', activation='relu')(x)
    
    tower_2 = Conv2D(nr_filters, (1, x.shape[2].value), padding='same', activation='relu')(x)
    tower_2 = Conv2D(nr_filters, (3, tower_2.shape[2].value), padding='same', activation='relu')(tower_2)
    
    tower_3 = Conv2D(nr_filters, (1, x.shape[2].value), padding='same', activation='relu')(x)
    tower_3 = Conv2D(nr_filters, (5, tower_3.shape[2].value), padding='same', activation='relu')(tower_3)
    
    tower_4 = MaxPooling2D((3, 1), strides=(1, 1), padding='same')(x)
    tower_4 = Conv2D(nr_filters, (1, tower_4.shape[2].value), padding='same', activation='relu')(tower_4)
    
    print(tower_1.shape)
    print(tower_2.shape)
    print(tower_3.shape)
    print(tower_4.shape)
    
    return concatenate([tower_1, tower_2, tower_3, tower_4], axis=3)


def ConvList(x):
    x = Conv2D(16, (3, x.shape[2].value), padding='valid', activation='relu')(x)
    print(x.shape)
    x = Conv2D(16, (3, x.shape[2].value), padding='valid', activation='relu')(x)
    print(x.shape)
    x = Conv2D(16, (3, x.shape[2].value), padding='valid', activation='relu')(x)
    print(x.shape)
    return x

In [219]:
def create_model(X, y):
    K.clear_session()

    windowsize = X.shape[1]
    feature_len = X.shape[2]

    inputs = Input((windowsize, feature_len, 1, ))
    x = inputs
    x = Conv2D(16, (1, x.shape[2].value), padding='valid', activation='relu')(x)
    x = InceptionModule2(x)
    #x = ConvList(x)
    x = MaxPooling2D((5, 1), strides=(2, 1), padding='valid')(x)
    x = Dropout(.1)(x)
    x = Flatten()(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(.3)(x)
    x = Dense(10, activation="relu")(x)
    x = Dense(y.shape[1], activation="softmax")(x)
    outputs = x

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


def train_model(model, X_train, y_train, epochs):
    
    tensorboard = kc.TensorBoard(log_dir="tensorboard/{}".format(time.strftime("%y-%m-%d_%H-%M-%S")), 
                                 write_images=True, histogram_freq = 1)
    
    #earlystopping = kc.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    model.fit(X_train, y_train, epochs=epochs, 
              validation_split=.2, 
              batch_size=32, 
              verbose=0,
              callbacks=[tensorboard])


def evaluate_model(model, X_test, y_test):
    loss, metric = model.evaluate(X_test,y_test)
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
    
    print("\nLoss: {} \nMetric: {}".format(loss, metric))
    print("\nConfusion Matrix:")
    for row in cm:
        print(row)

In [ ]:
X_train, X_test, y_train, y_test = get_data(["wells", "chesterton"])

In [ ]:
model = create_model(X_train, y_train)
train_model(model, X_train, y_train, epochs=20)
evaluate_model(model, X_test, y_test)

(?, 100, 1, 16)
(?, 100, 1, 16)
(?, 100, 1, 16)
(?, 100, 1, 16)


In [213]:
#model = create_model(X_train, y_train)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_54 (InputLayer)           (None, 100, 166, 1)  0                                            
__________________________________________________________________________________________________
conv2d_298 (Conv2D)             (None, 100, 1, 16)   2672        input_54[0][0]                   
__________________________________________________________________________________________________
conv2d_300 (Conv2D)             (None, 100, 1, 16)   272         conv2d_298[0][0]                 
__________________________________________________________________________________________________
conv2d_302 (Conv2D)             (None, 100, 1, 16)   272         conv2d_298[0][0]                 
__________________________________________________________________________________________________
max_poolin

In [216]:
kc.TensorBoard?

In [208]:
time.strftime("%y-%m-%d_%H-%M-%S")

'18-10-29_07-40-27'